In [1]:
import numpy as np

# Estimating income processes

Consider $N$ households indexed by $i$ who are in the labor market for $T$ periods indexed by $t \in \{1,2,\dots,T\}$.

Their **wage income** is stochastic and given by

$$ 
P_{i,t} = \psi_{i,t} P_{i,t-1},\\
\tilde{Y}_{i,t} = \xi_{i,t} P_{i,t},\\
Y_{i,t} = \begin{cases}
0 & \text{if } \mu_{i,t} < \pi\\
\tilde{Y}_{i,t} & \text{else}
\end{cases} \\
\psi_{i,t} \sim \text{LogNormal}(-0.5\sigma_{\psi}^2,\sigma_{\psi})\\ 
\xi_{i,t} \sim \text{LogNormal}(-0.5\sigma_{\xi}^2,\sigma_{\xi})\\
\mu_{i,t} \sim \text{Uniform}(0,1)\\
P_{0} = 1
$$

where 

- $\sigma_{\psi}$ is the standard deviation of the *permanent* shocks, $\psi_{i,t}$
- $\sigma_{\xi}$ is the standard deviation of the *transitory* shocks, $\xi_{i,t}$
- $\pi$ is the risk of unemployment.

**The data you have access to is:**

In [2]:
dataY = np.load('dataY.npy')
T,N = dataY.shape

**Question 1:** Calculate income growth rates as log-changes 

$$
\Delta \log(Y_{i,t}) \equiv \begin{cases}
\log(Y_{i,t})-\log(Y_{i,t-1}) & \text{if } Y_{i,t}>0 \text{ and } Y_{i,t-1}>0\\
\text{NaN} & \text{else}
\end{cases}
$$

where $\text{NaN}$ is *not-a-number* (i.e. `np.nan`).

**Question 2:** Calculate the following 3 statistics from the data

- $s_1^{data}$: Share of observations with $Y_{i,t} = 0$
- $s_2^{data}$: Variance of income growth rate, $\text{Var}(\Delta\log{Y_{i,t}})$
- $s_3^{data}$: Co-variance of income growth rates one period apart, $\text{Cov}(\Delta\log{Y_{i,t}},\Delta\log{Y_{i,t-1}})$

**Question 3:** Simulate the income process  using your own choice of $\sigma_{\psi}$, $\sigma_{\xi}$, $\pi$,  $T$ and $N$. Calculate the 3 same  statistics. Compare with the data statistics.

- $s_1^{sim}(\sigma_{\psi},\sigma_{\xi},\pi)$: Share of observations with $Y_{i,t} = 0$
- $s_2^{sim}(\sigma_{\psi},\sigma_{\xi},\pi)$: Variance of income growth $\text{Var}(\Delta\log{Y_{i,t}})$
- $s_3^{sim}(\sigma_{\psi},\sigma_{\xi},\pi)$: Co-variance of income growth one periods apart $\text{Cov}(\Delta\log{Y_{i,t}},\Delta\log{Y_{i,t-1}})$


**Question 4:** Solve the following minimization problem to estimate  $\sigma_{\psi}$, $\sigma_{\xi}$ and $\pi$

$$
\sigma_{\psi}^{\ast},\sigma_{\xi}^{\ast},\pi^{\ast} = \arg \min_{\sigma_{\psi}\geq0,\sigma_{\xi}\geq0,\pi\in[0,1]} (s_1^{sim}-s_1^{data})^2 + (s_2^{sim}-s_2^{data})^2 + (s_3^{sim}-s_3^{data})^2
$$

where for each new guess of $\sigma_{\psi}$, $\sigma_{\xi}$, and $\pi$ you should be re-simulating the data *with the same seed* and re-calculating the 3 statistics.

In [3]:
def objective():
    pass

# res = optimize.minimize(objective,x,method='L-BFGS-B',bounds=?,args=?,options={'eps':1e-4})
# hint: options={'eps':1e-4} uses a larger step-size when approximating the jacbian, which is useful in this case

# Wealth in the utility function

In the final period, $t=T$, the household solves the following problem

$$
\begin{aligned}
v_{T}(a_{T-1}) & = \max_{c_{T}} \frac{c_T^{1-\rho}}{1-\rho} + \kappa \frac{(a_T+\underline{a})^{1-\sigma}}{1-\sigma}
\\ & \text{s.t.} & \\
a_{T}& = (1+r)a_{T-1} + y - c_T
\end{aligned}
$$

where

- $a_t$ is end-of-period assets in period $t$
- $c_t$ is consumption in period $t$
- $\rho$ is the CRRA-coefficient for consumption utility
- $\sigma$ is the CRRA-coefficient for wealth utility
- $\underline{a}$ is an *additive* scaling factor for wealth utility
- $\kappa$ is a *multiplicative* scaling factor for wealth utility
- $r$ is the rate of return
- $y$ is income

The optimal **consumption function** is denoted $c_t^{\ast}(a_{t-1})$.

The optimal **savings function** is denoted $a_t^{\ast}(a_{t-1}) \equiv (1+r)a_{t-1} + y - c_t^{\ast}(a_{t-1})$.

In [4]:
# a. parameters
rho = 2.0
sigma = 1.2
kappa = 0.6
a_ubar = 2.0
r = 0.04
y = 1.0

# b. grids
a_lag_vec = np.linspace(0,300,300)

**Question 1:** Find and plot the functions $v_{T}(a_{T-1})$, $c_T^{\ast}(a_{T-1})$, and $a_T^{\ast}(a_{T-1})$.

In all periods *before the last*, $t < T$, the household solves:

$$
\begin{aligned}
v_{t}(a_{t-1}) & = \max_{c_{t}} \frac{c_t^{1-\rho}}{1-\rho} + \kappa \frac{(a_t+\underline{a})^{1-\sigma}}{1-\sigma} + \beta v_{t+1}(a_t)\\ & \text{s.t.} & \\
a_{t}& = (1+r)a_{t-1} + y - c_t
\end{aligned}
$$

where $\beta$ is the discount factor for future utility.

In [5]:
beta = 0.97
T = 20

**Question 2:** Find and plot $v_{T-1}(a_{T-2})$ and  $c_{T-1}^{\ast}(a_{T-2})$.

**Question 3:** Find $c_t^{\ast}(a_{t-1})$ for $t \in \{0,1,\dots,T\}$ and plot them in a single figure.

Define the saving rate as:

$$
s_t^{\ast}(a_{t-1}) \equiv \frac{a_t - a_{t-1}}{y+ra_{t-1}} = \frac{((1+r)a_{t-1} + y - c_t^{\ast}(a_{t-1})) - a_{T-1}}{y+ra_{t-1}}
$$

**Question 4:** Plot $s_0^{\ast}(a_{-1})$. Do the rich or the poor save the most?

**Question 5:** Can you change the parameter choices such that $s_0^{\ast}(a_{-1})$ is monotonically decreasing in $a_{-1}$?

# Refined grid search

Let $\boldsymbol{x} = \left[\begin{array}{c}
x_1 \\
x_2\\
\end{array}\right]$ be a two-dimensional vector.

Consider the following algorithm:

**Algorithm:** `grid_search()`

**Goal:** Minimize the function $f(\boldsymbol{x})$.

1. Choose a grid size $N$ and minimum and maximum values of $x_1$ and $x_2$ denoted $\overline{x}_1 > \underline{x}_1$ and $\overline{x}_2 > \underline{x}_2$
2. Calculate step-sizes 

    $$
    \Delta_1 = (\overline{x}_1 - \underline{x}_1)/(N-1) \\
    \Delta_2 = (\overline{x}_2 - \underline{x}_2)/(N-1)
    $$
    
3. Find the grid point with the lowest function value by solving

    $$
    j_1^{\ast},j_2^{\ast} = \arg \min_{j_1\in\{0,...N-1\},j_2\in\{0,...N-1\}} f(\underline{x}_1 + j_1\Delta_1, \underline{x}_2 + j_2\Delta_2)
    $$

 

4. Return $x_1^{\ast} = \underline{x}_1 + j_1^{\ast}\Delta_1$, $x_2^{\ast} = \underline{x}_2 + j_2^{\ast}\Delta_2$ and $f^{\ast} = f(x_1^{\ast},x_2^{\ast})$.

**Question 1:** Implement the `grid_search()` algorithm to minimize the rosen function.

> **Hint:** The global minimum of the rosen function is $0$ at $(1,1)$.

In [6]:
def rosen(x):
    return (1.0-x[0])**2+2*(x[1]-x[0]**2)**2

In [7]:
def grid_search(f,x1_min,x1_max,x2_min,x2_max,N):
    return np.nan,np.nan,np.nan

# settings
x1_min = 0
x1_max = 5
x2_min = 0
x2_max = 4
N = 100

# apply grid search
x1,x2,f = grid_search(rosen,x1_min,x1_max,x2_min,x2_max,N)
print(f'minimum found at ({x1:.8f},{x2:.8f}) with the function value {f:.8f}')

minimum found at (nan,nan) with the function value nan


Also, consider the following algorithm:

**Algorithm:** `refined_grid_search()`

**Goal:** Minimize the function $f(\boldsymbol{x})$.

1. Choose a grid size $N$ and minimum and maximum values of $x_1$ and $x_2$ denoted $\overline{x}_1 > \underline{x}_1$ and $\overline{x}_2 > \underline{x}_2$, and a refinement-level $K$

2. Set $k=0$

3. If $k > 0$: Update the minimum and maximum values by

    $$
    \tilde{\Delta}_1 = 3(\overline{x}_1 - \underline{x}_1)/(N-1) \\
    \tilde{\Delta}_2 = 3(\overline{x}_2 - \underline{x}_2)/(N-1) \\
    \underline{x}_1 = \max(\underline{x}_1,x_1^{\ast}-\tilde{\Delta}_1) \\
    \underline{x}_2 = \max(\underline{x}_2,x_2^{\ast}-\tilde{\Delta}_2) \\
    \overline{x}_1 = \min(\overline{x}_1,x_1^{\ast}+\tilde{\Delta}_1) \\
    \overline{x}_2 = \min(\overline{x}_2,x_2^{\ast}+\tilde{\Delta}_2)
    $$
    
4. Apply the `grid_search()` algorithm returning $x_1^{\ast}$, $x_2^{\ast}$ and $f^\ast$ 

5. Increment $k$ by one

6. If $k < K$ return to step 3 else continue

7. Return  $x_1^{\ast}$, $x_2^{\ast}$ and $f^{\ast}$


**Question 2:** Implement the `refined_grid_search()` algorithm to minimize the rosen function.

In [8]:
def refined_grid_search(f,x1_min,x1_max,x2_min,x2_max,N,K):
    return np.nan,np.nan,np.nan

# more settings
K = 10

# apply refined grid search
x1,x2,f = refined_grid_search(rosen,x1_min,x1_max,x2_min,x2_max,N,K)
print(f'minimum found at ({x1:.8f},{x2:.8f}) with the function value {f:.8f}')

minimum found at (nan,nan) with the function value nan
